In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import time
import warnings
warnings.filterwarnings('ignore')

enc = OneHotEncoder()
path =  '../web/dataset/'
train = pd.read_csv(path+'train.csv')
vali = pd.read_csv(path+'validation.csv')
test = pd.read_csv(path+'test.csv')
start_time = time.time()

In [2]:
basectr = 0.0007539649884458758#train['click'].value_counts(normalize = True)[1]

In [3]:
ad_ctr = [0.00083473226564105031,
 0.00050464562342478474,
 0.00074884754565216914,
 0.00076448084204447765,
 0.00030658545558598702,
 0.00062225602378400803,
 0.0007180766777520589,
 0.0046066034099877034,
 0.00030675100937663219]#[]
ad_name = [1458,3476,3427,3358,2259,2821,3386,2997,2261]
# for i in ad_name:
#     ad_ctr.append(train['click'][train['advertiser']==i].value_counts(normalize = True)[1])
ad_len = len(ad_name)

In [4]:
train_onehot = pd.read_csv(path+'train_onehot.csv')
vali_onehot = pd.read_csv(path+'vali_onehot.csv')

In [5]:
import pickle
# from sklearn.linear_model import Lasso
# lasso_model = pickle.load(open('lasso_model.sav','rb'))
rf_model = pickle.load(open('rf_model.sav','rb'))

In [6]:
names =[str(x) for x in list(vali_onehot.columns)]
importance = pd.DataFrame({'Importance0':rf_model[0].feature_importances_,'Importance1':rf_model[1].feature_importances_,'Importance2':rf_model[2].feature_importances_,
             'Importance3':rf_model[3].feature_importances_,'Importance4':rf_model[4].feature_importances_,
              'Importance5':rf_model[5].feature_importances_,'Importance6':rf_model[6].feature_importances_,
              'Importance7':rf_model[7].feature_importances_,'Importance8':rf_model[8].feature_importances_})

In [7]:
importance['importance_sum'] = importance.apply(lambda x: x.sum(),axis=1)
importance['Names'] = names

In [8]:
vali_ad_onehot = []
for i in ad_name:
    vali_ad_onehot.append(vali_onehot[vali_onehot["advertiser_"+str(i)]==1]) 

In [10]:
def dataprocessing (data,data_onehot):
    slotprice = data['slotprice']
    bidprice = data['bidprice']
    payprice = data['payprice']
    click = data['click']
    
    ad_onehot = []
    
    ad_onehot = []
    for i in ad_name:
        ad_onehot.append(data_onehot[data_onehot["advertiser_"+str(i)]==1]) 
    
    ad_click = []
    for i in ad_name:
        ad_click.append(data['click'][data['advertiser']==i])
        
    return ad_onehot, ad_click, slotprice, bidprice, payprice

In [11]:
train_ad_onehot,train_ad_click,train_slotprice,train_bidprice,train_payprice = dataprocessing(train,train_onehot)
vali_ad_onehot,vali_ad_click,vali_slotprice,vali_bidprice,vali_payprice = dataprocessing(vali,vali_onehot)

In [12]:
##########

In [13]:
train_click = train[['click','advertiser']]
vali_ad_onehot_xgb= train_onehot.drop(remove_list,axis=1)
train_ad_onehot_xgb= vali_onehot.drop(remove_list,axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_onehot, train_click, test_size=0.2, random_state=42)

In [14]:
def dataprocessing_ (data_onehot,click):
    ad_onehot = []
    for i in ad_name:
        ad_onehot.append(data_onehot[data_onehot["advertiser_"+str(i)]==1]) 
#     print('done')
    ad_click = []
    for i in ad_name:
#         print(type(click))
        a = click[click['advertiser']==i]
        a = a.drop('advertiser',1)
#         print(a.head())
        ad_click.append(a)    
    return ad_onehot, ad_click

In [ ]:
_train_ad_onehot,_train_ad_click = dataprocessing_(X_train,y_train)
_ad_onehot,_ad_click= dataprocessing_(X_test,y_test)

In [ ]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.3.0-posix-seh-rt_v5-rev1\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH'] 
import xgboost as xgb

xgb_model = []
train_ad_pd = []
train_ad_rmse = []
train_ad_auc = []
train_ad_report = []
vali_ad_pd = []
vali_ad_rmse = []
vali_ad_auc = []

param = {'max_depth':4, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
plst = param.items()

num_round = 10

for i in range(ad_len):
    dtrain = (xgb.DMatrix(_train_ad_onehot[i],_train_ad_click[i]))
    dvali = (xgb.DMatrix(_ad_onehot[i],_ad_click[i]))
    evallist  = [(dvali,'eval'), (dtrain,'train')]
    dptrain = xgb.DMatrix(train_ad_onehot[i])
    dpvali =xgb.DMatrix(vali_ad_onehot[i])
    
    xgb_model.append(xgb.train( plst, dtrain, num_round, evallist ))
    train_ad_pd.append(xgb_model[i].predict(dptrain))
    train_ad_rmse.append(rmse(train_ad_click[i],train_ad_pd[i]))
    train_ad_auc.append(auc(train_ad_click[i],train_ad_pd[i]))
    
    vali_ad_pd.append(xgb_model[i].predict(dpvali))
    vali_ad_rmse.append(rmse(vali_ad_click[i],vali_ad_pd[i]))
    vali_ad_auc.append(auc(vali_ad_click[i],vali_ad_pd[i]))
    print(i+1,'/9')

In [ ]:
#####

In [35]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import classification_report as report

In [26]:
train_ad_onehot[0].shape

(540293, 280)

In [32]:
for i in range(len(train_ad_onehot)):
    vali_ad_onehot[i] = vali_ad_onehot[i].drop(remove_list,axis=1)
    train_ad_onehot[i] = train_ad_onehot[i].drop(remove_list,axis=1)

In [33]:
train_ad_onehot[0].shape

(540293, 259)

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf_model = []
train_ad_rfpd = []
train_ad_rfrmse = []
train_ad_rfauc = []
vali_ad_rfpd = []
vali_ad_rfrmse = []
vali_ad_rfauc = []

for i in range(ad_len):
    rf_model.append(RandomForestClassifier(n_estimators=50, max_depth=15, random_state=2017))
    rf_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_rfpd.append((rf_model[i].predict_proba(train_ad_onehot[i]))[:,1])
    train_ad_rfrmse.append(rmse(train_ad_click[i],train_ad_rfpd[i]))
    train_ad_rfauc.append(auc(train_ad_click[i],train_ad_rfpd[i]))
    vali_ad_rfpd.append((rf_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_rfrmse.append(rmse(vali_ad_click[i],vali_ad_rfpd[i]))
    vali_ad_rfauc.append(auc(vali_ad_click[i],vali_ad_rfpd[i]))
    print(i+1,'/9')
    

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [ ]:
j = 0
print('random forest validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_rfrmse[j],'AUC',vali_ad_rfauc[j])
    j+=1

In [38]:
best = [0.964258274281,0.823949256842,0.952029031574,0.937509717487,0.644292141306,0.665704540131,0.729408271574,0.661435393834,0.773310886644]
list(map(lambda x: x[0]-x[1], zip(best, vali_ad_rfauc)))#0.00005,0.00003#0.00004

[0.047720050020891636,
 0.083414563595206603,
 0.048498313554645622,
 0.046871133535579701,
 0.11159467665461043,
 0.036810489537620894,
 0.056729395361958312,
 0.029707185291543703,
 0.36625514403270187]

In [42]:
from sklearn.linear_model import ElasticNet

et_model = []
train_ad_etpd = []
train_ad_etrmse = []
train_ad_etauc = []
vali_ad_etpd = []
vali_ad_etrmse = []
vali_ad_etauc = []

for i in range(ad_len):
    et_model.append(ElasticNet(alpha=0.000008, l1_ratio=0.7))
    et_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_etpd.append((et_model[i].predict(train_ad_onehot[i])))
    train_ad_etrmse.append(rmse(train_ad_click[i],train_ad_etpd[i]))
    train_ad_etauc.append(auc(train_ad_click[i],train_ad_etpd[i]))
    vali_ad_etpd.append((et_model[i].predict(vali_ad_onehot[i])))
    vali_ad_etrmse.append(rmse(vali_ad_click[i],vali_ad_etpd[i]))
    vali_ad_etauc.append(auc(vali_ad_click[i],vali_ad_etpd[i]))
    print(i+1,'/9')

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [43]:
j = 0
print('elastic net validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_etrmse[j],'AUC',vali_ad_etauc[j])
    j+=1

elastic net validation results
ad 1458 : RMSE 0.0242139776263 AUC 0.907493122134
ad 3476 : RMSE 0.0182912419726 AUC 0.774192749506
ad 3427 : RMSE 0.029686666166 AUC 0.90139639401
ad 3358 : RMSE 0.0277666099243 AUC 0.879664119864
ad 2259 : RMSE 0.0258762792391 AUC 0.656032534019
ad 2821 : RMSE 0.0249766780795 AUC 0.645225396237
ad 3386 : RMSE 0.0244067778075 AUC 0.72017722683
ad 2997 : RMSE 0.0654756718892 AUC 0.628975468606
ad 2261 : RMSE 0.0193369359226 AUC 0.586464646465


In [46]:
best = [0.964258274281,0.823949256842,0.952029031574,0.937509717487,0.656032534019,0.665704540131,0.729408271574,0.661435393834,0.773310886644]
list(map(lambda x: x[0]-x[1], zip(best, vali_ad_etauc)))#0.00005,0.00003#0.00004

[0.056765152146777353,
 0.049756507336315869,
 0.050632637563969629,
 0.057845597623033873,
 1.1779466291272911e-13,
 0.020479143894272966,
 0.0092310447437630172,
 0.032459925228038755,
 0.18684624017935358]

In [47]:
from sklearn.ensemble import ExtraTreesClassifier
etr_model = []
train_ad_etrpd = []
train_ad_etrrmse = []
train_ad_etrauc = []
vali_ad_etrpd = []
vali_ad_etrrmse = []
vali_ad_etrauc = []

for i in range(ad_len):
    etr_model.append(ExtraTreesClassifier(n_estimators=150, max_depth=20, random_state=2017))
    etr_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_etrpd.append((etr_model[i].predict_proba(train_ad_onehot[i]))[:,1])
    train_ad_etrrmse.append(rmse(train_ad_click[i],train_ad_etrpd[i]))
    train_ad_etrauc.append(auc(train_ad_click[i],train_ad_etrpd[i]))
    vali_ad_etrpd.append((etr_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_etrrmse.append(rmse(vali_ad_click[i],vali_ad_etrpd[i]))
    vali_ad_etrauc.append(auc(vali_ad_click[i],vali_ad_etrpd[i]))
    print(i+1,'/9')


1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [51]:
j = 0
print('extra tree classifier validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_etrrmse[j],'AUC',vali_ad_etrauc[j])
    j+=1

extra tree classifier validation results
ad 1458 : RMSE 0.0235388070982 AUC 0.927114297624
ad 3476 : RMSE 0.0182583882036 AUC 0.763425777334
ad 3427 : RMSE 0.0273934247314 AUC 0.930034391114
ad 3358 : RMSE 0.0256652074792 AUC 0.892944502476
ad 2259 : RMSE 0.0259940178264 AUC 0.634499246487
ad 2821 : RMSE 0.0255808393772 AUC 0.673443843691
ad 3386 : RMSE 0.0246971551726 AUC 0.65311591707
ad 2997 : RMSE 0.066202102052 AUC 0.611466762266
ad 2261 : RMSE 0.0194621554121 AUC 0.638862701085


In [50]:
best = [0.964258274281,0.823949256842,0.952029031574,0.937509717487,0.656032534019,0.674527150999,0.729408271574,0.661435393834,0.773310886644]
list(map(lambda x: x[0]-x[1], zip(best, vali_ad_etrauc)))#0.00005,0.00003#0.00004

[0.037143976656989985,
 0.060523479508016953,
 0.021994640460063386,
 0.044565215011367676,
 0.021533287531698808,
 0.0010833073075571242,
 0.076292354503983306,
 0.04996863156830289,
 0.13444818555907678]

In [19]:
# merge the single model
def ad_to_total (ad_onehot,ad_pd):
    ad_pctr = ad_onehot
    j = 0
    for i in ad_name:
        ad_pctr[j]['pctr'] = ad_pd[j]
        ad_pctr[j]['ad'] = i
        ad_pctr[j] = ad_pctr[j][['pctr','ad']]
        j+=1
    pctr = (pd.concat(ad_pctr)).sort_index()
    return pctr

In [52]:
from sklearn.naive_bayes import GaussianNB
gnb_model = []
train_ad_gnbpd = []
train_ad_gnbrmse = []
train_ad_gnbauc = []
vali_ad_gnbpd = []
vali_ad_gnbrmse = []
vali_ad_gnbauc = []

for i in range(ad_len):
    gnb_model.append(GaussianNB())
    gnb_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_gnbpd.append((gnb_model[i].predict_proba(train_ad_onehot[i]))[:,1])
    train_ad_gnbrmse.append(rmse(train_ad_click[i],train_ad_gnbpd[i]))
    train_ad_gnbauc.append(auc(train_ad_click[i],train_ad_gnbpd[i]))
    vali_ad_gnbpd.append((gnb_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_gnbrmse.append(rmse(vali_ad_click[i],vali_ad_gnbpd[i]))
    vali_ad_gnbauc.append(auc(vali_ad_click[i],vali_ad_gnbpd[i]))
    print(i+1,'/9')

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [53]:
j = 0
print('GaussianNB validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_gnbrmse[j],'AUC',vali_ad_gnbauc[j])
    j+=1

GaussianNB validation results
ad 1458 : RMSE 0.976379070427 AUC 0.513739057941
ad 3476 : RMSE 0.947188059746 AUC 0.474369871101
ad 3427 : RMSE 0.979316400419 AUC 0.514837624497
ad 3358 : RMSE 0.966329912104 AUC 0.516224096739
ad 2259 : RMSE 0.769373882783 AUC 0.567561278312
ad 2821 : RMSE 0.940847088705 AUC 0.527574318785
ad 3386 : RMSE 0.967382830784 AUC 0.50357150901
ad 2997 : RMSE 0.960782191562 AUC 0.517303851275
ad 2261 : RMSE 0.66423765921 AUC 0.579386457164


In [54]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis

lda_model = []
train_ad_ldapd = []
train_ad_ldarmse = []
train_ad_ldaauc = []
vali_ad_ldapd = []
vali_ad_ldarmse = []
vali_ad_ldaauc = []

for i in range(ad_len):
    lda_model.append(LinearDiscriminantAnalysis())
    lda_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_ldapd.append((lda_model[i].predict_proba(train_ad_onehot[i]))[:,1])
    train_ad_ldarmse.append(rmse(train_ad_click[i],train_ad_ldapd[i]))
    train_ad_ldaauc.append(auc(train_ad_click[i],train_ad_ldapd[i]))
    vali_ad_ldapd.append((lda_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_ldarmse.append(rmse(vali_ad_click[i],vali_ad_ldapd[i]))
    vali_ad_ldaauc.append(auc(vali_ad_click[i],vali_ad_ldapd[i]))
    print(i+1,'/9')

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [55]:
j = 0
print('lda validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_ldarmse[j],'AUC',vali_ad_ldaauc[j])
    j+=1

lda validation results
ad 1458 : RMSE 0.0346338015275 AUC 0.911770237599
ad 3476 : RMSE 0.0444493767818 AUC 0.776424600486
ad 3427 : RMSE 0.0585734178868 AUC 0.89698890125
ad 3358 : RMSE 0.112242940798 AUC 0.877683394978
ad 2259 : RMSE 0.0698476230509 AUC 0.593222264084
ad 2821 : RMSE 0.0331032494778 AUC 0.661983721112
ad 3386 : RMSE 0.0809628476998 AUC 0.704859537492
ad 2997 : RMSE 0.0654784807177 AUC 0.627394243573
ad 2261 : RMSE 0.0394046277432 AUC 0.57418630752


In [93]:
from sklearn.neighbors import KNeighborsClassifier
knc_model = []
train_ad_kncpd = []
train_ad_kncrmse = []
train_ad_kncauc = []
vali_ad_kncpd = []
vali_ad_kncrmse = []
vali_ad_kncauc = []

for i in range(ad_len):
    knc_model.append(KNeighborsClassifier(1))
    knc_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_kncpd.append((knc_model[i].predict_proba(train_ad_onehot[i]))[:,1])
    train_ad_kncrmse.append(rmse(train_ad_click[i],train_ad_kncpd[i]))
    train_ad_kncauc.append(auc(train_ad_click[i],train_ad_kncpd[i]))
    vali_ad_kncpd.append((knc_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_kncrmse.append(rmse(vali_ad_click[i],vali_ad_kncpd[i]))
    vali_ad_kncauc.append(auc(vali_ad_click[i],vali_ad_kncpd[i]))
    print(i+1,'/9')

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [94]:
j = 0
print('knc validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_kncrmse[j],'AUC',vali_ad_kncauc[j])
    j+=1

knc validation results
ad 1458 : RMSE 0.99939173504 AUC 0.460266777824
ad 3476 : RMSE 0.500199501897 AUC 0.576781419271
ad 3427 : RMSE 0.500416650205 AUC 0.666338869464
ad 3358 : RMSE 0.500531428215 AUC 0.499645243304
ad 2259 : RMSE 0.5 AUC 0.5
ad 2821 : RMSE 0.5 AUC 0.5
ad 3386 : RMSE 0.500642749421 AUC 0.529867956068
ad 2997 : RMSE 0.0656423349818 AUC 0.5
ad 2261 : RMSE 0.499906498363 AUC 0.500187055743


In [33]:
from sklearn.ensemble import AdaBoostClassifier
adb_model = []
train_ad_adbpd = []
train_ad_adbrmse = []
train_ad_adbauc = []
vali_ad_adbpd = []
vali_ad_adbrmse = []
vali_ad_adbauc = []

for i in range(ad_len):
    adb_model.append(AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=2017))
    adb_model[i].fit(train_ad_onehot[i], train_ad_click[i])
#     train_ad_adbpd.append((adb_model[i].predict_proba(train_ad_onehot[i]))[:,1])
#     train_ad_adbrmse.append(rmse(train_ad_click[i],train_ad_adbpd[i]))
#     train_ad_adbauc.append(auc(train_ad_adbpd[i],train_ad_adbpd[i]))
    vali_ad_adbpd.append((adb_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_adbrmse.append(rmse(vali_ad_click[i],vali_ad_adbpd[i]))
    vali_ad_adbauc.append(auc(vali_ad_click[i],vali_ad_adbpd[i]))
    print(i+1,'/9')


1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [33]:
# j = 0
# print('Adaboosting trainning results')
# for i in ad_name:
#     print('ad',i,':','RMSE',train_ad_adbrmse[j],'AUC',train_ad_adbauc[j])
#     j+=1
j = 0
print('Adaboosting validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_adbrmse[j],'AUC',vali_ad_adbauc[j])
    j+=1

Adaboosting validation results
ad 1458 : RMSE 0.450369772464 AUC 0.963908628595
ad 3476 : RMSE 0.449349903403 AUC 0.674023949059
ad 3427 : RMSE 0.453485636704 AUC 0.942543220668
ad 3358 : RMSE 0.449437385353 AUC 0.93714401151
ad 2259 : RMSE 0.416637456771 AUC 0.592787332122
ad 2821 : RMSE 0.452913962076 AUC 0.591012453154
ad 3386 : RMSE 0.459870000249 AUC 0.665648746772
ad 2997 : RMSE 0.466973307183 AUC 0.630300624808
ad 2261 : RMSE 0.403631403828 AUC 0.569726898616


In [ ]:
etr_model[1].feature_importances_  
model = SelectFromModel(etr_model, prefit=True)
X_new = model.transform(train_ad_onehot[1])
X_new_test = model.transform(vali_ad_onehot[1])
clf_ET2 = rf_model[1]#ExtraTreesRegressor(n_estimators=150, max_depth=20, random_state=2017)
t = time.time()
clf_ET2.fit(X_new, train_ad_click[1])
print('model training time:',round((time.time()-t)/60,3) ,'minutes\n')
t = time.time()
y_pred_ET2 = clf_ET2.predict(X_new_test)
print('predicting time:',round((time.time()-t)/60,3) ,'minutes\n')

In [105]:
import statsmodels.formula.api as sm 
result = sm.ols(formula = 'train.click ~ et_pctr_tr.pctr + rf_pctr_tr.pctr ', data = train_onehot).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            train.click   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                 4.974e+06
Date:                Tue, 11 Apr 2017   Prob (F-statistic):               0.00
Time:                        08:03:55   Log-Likelihood:             7.9556e+06
No. Observations:             2697738   AIC:                        -1.591e+07
Df Residuals:                 2697735   BIC:                        -1.591e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        6.491e-05   7.78e-06      8.347      0.000    4.97e-05    8.02e-05
et_pctr_tr.pctr     1.0600      0.001   2014.837      0.000       1.059       1.061
rf_pctr_tr.pctr    -0.1476      0.002    -89.809      0.000      -0.151      -0.144
==============================================================================
Omnibus:                  9083823.659   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):    2445764167204.572
Skew:                          61.005   Prob(JB):                         0.00
Kurtosis:                    4665.989   Cond. No.                         220.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [106]:
scaled =(et_pctr_tr*1.06-0.1476*rf_pctr_tr+6.491e-05)

In [107]:
rmse(train.click,scaled.pctr),auc(train.click,scaled.pctr)

(0.012677687788632598, 0.99968561544259016)

In [111]:
rmse(train.click,rf_pctr_tr.pctr),auc(train.click,rf_pctr_tr.pctr)

(0.022936234989245914, 0.99644619443890226)

In [108]:
rmse(train.click,et_pctr_tr.pctr),auc(train.click,et_pctr_tr.pctr)

(0.012708231345303328, 0.99968022812427337)

In [114]:
import statsmodels.formula.api as sm 
result = sm.ols(formula = 'vali.click ~ et_pctr.pctr + rf_pctr.pctr ', data = vali_onehot).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             vali.click   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                 2.334e+04
Date:                Tue, 11 Apr 2017   Prob (F-statistic):               0.00
Time:                        08:10:09   Log-Likelihood:             6.7411e+05
No. Observations:              299749   AIC:                        -1.348e+06
Df Residuals:                  299746   BIC:                        -1.348e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.0006   4.72e-05    -13.088      0.000      -0.001      -0.001
et_pctr.pctr     0.0567      0.003     18.298      0.000       0.051       0.063
rf_pctr.pctr     1.7761      0.012    152.541      0.000       1.753       1.799
==============================================================================
Omnibus:                   803245.063   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      20567924744.227
Skew:                          32.901   Prob(JB):                         0.00
Kurtosis:                    1284.593   Cond. No.                         253.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [115]:
scaled_vali =(et_pctr+rf_pctr)/2

In [116]:
rmse(vali.click,scaled_vali.pctr),auc(vali.click,scaled_vali.pctr)

(0.026409143746440905, 0.83312295458333319)

In [117]:
rmse(vali.click,et_pctr.pctr),auc(vali.click,et_pctr.pctr)

(0.029333945865646645, 0.74721666594427905)

In [118]:
rmse(vali.click,rf_pctr.pctr),auc(vali.click,rf_pctr.pctr)

(0.025991713806223657, 0.83314297166122453)

In [16]:
from sklearn.linear_model import Ridge

ridge_model = []
train_ad_ridgepd = []
train_ad_ridgermse = []
train_ad_ridgeauc = []
vali_ad_ridgepd = []
vali_ad_ridgermse = []
vali_ad_ridgeauc = []

for i in range(ad_len):
    ridge_model.append(Ridge(alpha=0.00004))
    ridge_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_ridgepd.append((ridge_model[i].predict(train_ad_onehot[i])))
    train_ad_ridgermse.append(rmse(train_ad_click[i],train_ad_ridgepd[i]))
    train_ad_ridgeauc.append(auc(train_ad_click[i],train_ad_ridgepd[i]))
    vali_ad_ridgepd.append((ridge_model[i].predict(vali_ad_onehot[i])))
    vali_ad_ridgermse.append(rmse(vali_ad_click[i],vali_ad_ridgepd[i]))
    vali_ad_ridgeauc.append(auc(vali_ad_click[i],vali_ad_ridgepd[i]))
    print(i+1,'/9')

MemoryError: 

In [ ]:
# j = 0
# print('Adaboosting trainning results')
# for i in ad_name:
#     print('ad',i,':','RMSE',train_ad_adbrmse[j],'AUC',train_ad_adbauc[j])
#     j+=1
j = 0
print('Ridge validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_ridgermse[j],'AUC',vali_ad_ridgeauc[j])
    j+=1

In [17]:
best = [0.964258274281,0.823949256842,0.952029031574,0.937509717487,0.644292141306,0.665704540131,0.729408271574,0.661435393834,0.773310886644]

In [27]:
best = [0.964258274281,0.823949256842,0.952029031574,0.937509717487,0.644292141306,0.665704540131,0.729408271574,0.661435393834,0.773310886644]
list(map(lambda x: x[0]-x[1], zip(best, vali_ad_ridgeauc)))#0.00005,0.00003#0.00004

[0.071949312213471783,
 0.032110718828218587,
 0.044672467683345207,
 0.041687744032436136,
 0.0094826248831707538,
 0.060298007104766294,
 0.034983374341061135,
 0.033842696917068649,
 -0.068275346052991792]

In [39]:
from sklearn.linear_model import Lasso

lasso_model = []
train_ad_lassopd = []
train_ad_lassormse = []
train_ad_lassoauc = []
vali_ad_lassopd = []
vali_ad_lassormse = []
vali_ad_lassoauc = []

for i in range(ad_len):
    lasso_model.append(Lasso(alpha=0.00004))
    lasso_model[i].fit(train_ad_onehot[i], train_ad_click[i])
    train_ad_lassopd.append((lasso_model[i].predict(train_ad_onehot[i])))
    train_ad_lassormse.append(rmse(train_ad_click[i],train_ad_lassopd[i]))
    train_ad_lassoauc.append(auc(train_ad_click[i],train_ad_lassopd[i]))
    vali_ad_lassopd.append((lasso_model[i].predict(vali_ad_onehot[i])))
    vali_ad_lassormse.append(rmse(vali_ad_click[i],vali_ad_lassopd[i]))
    vali_ad_lassoauc.append(auc(vali_ad_click[i],vali_ad_lassopd[i]))
    print(i+1,'/9')

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [40]:
# j = 0
# print('Adaboosting trainning results')
# for i in ad_name:
#     print('ad',i,':','RMSE',train_ad_adbrmse[j],'AUC',train_ad_adbauc[j])
#     j+=1
j = 0
print('Lasso validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_lassormse[j],'AUC',vali_ad_lassoauc[j])
    j+=1

Lasso validation results
ad 1458 : RMSE 0.0243192478018 AUC 0.889510462693
ad 3476 : RMSE 0.018284800259 AUC 0.801699099335
ad 3427 : RMSE 0.0297079952638 AUC 0.908316777919
ad 3358 : RMSE 0.0278228580201 AUC 0.899746195672
ad 2259 : RMSE 0.0258764216646 AUC 0.644292141306
ad 2821 : RMSE 0.0249759546687 AUC 0.579150970097
ad 3386 : RMSE 0.0244193769057 AUC 0.690008783885
ad 2997 : RMSE 0.0654709532679 AUC 0.611073056438
ad 2261 : RMSE 0.0193342687271 AUC 0.773310886644


In [23]:
best = [0.964258274281,
        0.823949256842,
        0.952029031574,
        0.937509717487,
        0.636482757856 ,
        0.665704540131,
        0.729408271574,
        0.661435393834,
        0.672383090161]

In [35]:
list(map(lambda x: x[0]-x[1], zip(best, vali_ad_lassoauc)))#0.00005,0.00003#0.00004

[0.074747811588211199,
 0.022250157507295643,
 0.043712253654949351,
 0.037763521814593637,
 -0.0078093834497932058,
 0.086553570034185512,
 0.039399487688886015,
 0.050362337396429591,
 -0.10092779648322003]

In [18]:
# import pickle
# pickle.dump(lasso_model,open('lasso_model.sav','wb'))

In [19]:
from sklearn.linear_model import Lasso
lasso_model = pickle.load(open('lasso_model.sav','rb'))

In [ ]:
# LR validation results
# ad 1458 : RMSE 0.240054189116 AUC 0.951993664027
# ad 3476 : RMSE 0.350240830746 AUC 0.659825493622
# ad 3427 : RMSE 0.295330816321 AUC 0.919187283676
# ad 3358 : RMSE 0.265198623833 AUC 0.898056721654
# ad 2259 : RMSE 0.337446378231 AUC 0.659415030362
# ad 2821 : RMSE 0.430764933074 AUC 0.614430434104
# ad 3386 : RMSE 0.437116856603 AUC 0.656655289915
# ad 2997 : RMSE 0.480052104172 AUC 0.61804773123
# ad 2261 : RMSE 0.2872565263 AUC 0.53164234942

In [20]:
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
pipe_model = []

for i in range(ad_len):
    pipe_model.append(Pipeline([('feature_selection', SelectFromModel(lasso_model[i])),('classification',rf_model[i])]))
    pipe_model[i].fit(train_ad_onehot[i],train_ad_click[i])

MemoryError: 

In [ ]:
pipe_pd = []
pipe_rmse = []
pipe_auc = []
for i in range(ad_len):
    
    pipe_pd.append((pipe_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    pipe_rmse.append(rmse(vali_ad_click[i],pipe_pd[i]))
    pipe_auc.append(auc(vali_ad_click[i],pipe_pd[i]))

In [ ]:
j = 0
print('pipe+rf validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',pipe_rmse[j],'AUC',pipe_auc[j])
    j+=1

In [60]:
j = 0
print('pipe+rf validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',pipe_rmse[j],'AUC',pipe_auc[j])
    j+=1

pipe+rf validation results
ad 1458 : RMSE 0.769013785999 AUC 0.650003001251
ad 3476 : RMSE 0.68768011187 AUC 0.615230079764
ad 3427 : RMSE 0.473224747424 AUC 0.688287596242
ad 3358 : RMSE 0.344162001414 AUC 0.719114268884
ad 2259 : RMSE 0.343447953261 AUC 0.499238176499
ad 2821 : RMSE 0.69972076961 AUC 0.501756714553
ad 3386 : RMSE 0.503582203109 AUC 0.543935905655
ad 2997 : RMSE 0.428160691195 AUC 0.490307794735
ad 2261 : RMSE 0.347741798059 AUC 0.499812944257


# Save Results

In [65]:
files = ['xgb_result','rf_result','rf_bag_result','etc_result','etc_bag_result','gbc_result','gbc_bag_result','adb_result','adb_bag_result']

In [66]:
def result_save (pd, auc, rmse):
    result = []
    result.append(pd)
    result.append(auc)
    result.append(rmse)
    
    return result

In [67]:
#bag_model = [rf_bag, etr_bag, gbc_bag, adb_bag] 

xgb_result = result_save (vali_ad_pd, vali_ad_auc, vali_ad_rmse,)
rf_result = result_save (vali_ad_rfpd, vali_ad_rfauc, vali_ad_rfrmse)
rf_bag_result = result_save (bag_prob[0],bag_auc[0],bag_rmse[0])
etc_result = result_save (vali_ad_etrpd, vali_ad_etrauc, vali_ad_etrrmse)
etc_bag_result = result_save (bag_prob[1],bag_auc[1],bag_rmse[1])
gbc_result = result_save (vali_ad_gbcpd, vali_ad_gbcauc, vali_ad_gbcrmse)
gbc_bag_result = result_save (bag_prob[2],bag_auc[2],bag_rmse[2])
adb_result = result_save (vali_ad_rfpd, vali_ad_rfauc, vali_ad_rfrmse)
adb_bag_result = result_save (bag_prob[3],bag_auc[3],bag_rmse[3])

In [71]:
result_list = [xgb_result,rf_result,rf_bag_result,etc_result,etc_bag_result,gbc_result,gbc_bag_result,adb_result,adb_bag_result]

In [73]:
import pickle
j = 0
for i in files:
    with open(i+".txt", "wb") as fp:
        pickle.dump(result_list[j], fp)
    j+=1